In [15]:
import pandas as pd

In [16]:
import csv

# file_path = 'data/rum_generated.csv'
df = pd.read_csv("../data/rum_generated.csv")
df.head(10)



,user_id,country,user_action,trainer_name,timestamp,readable_timestamp
0,1472,Egypt,sent_message,NaN,1755502288000,2025-08-18 08:31:28 AM
1,2811,Ghana,visit_home,NaN,1755991665000,2025-08-24 12:27:45 AM
2,9526,Ghana,visit_home,NaN,1751924316000,2025-07-07 10:38:36 PM
3,9900,Nigeria,visit_home,NaN,1758533458000,2025-09-22 10:30:58 AM
4,8784,Nigeria,view_plan,NaN,1751485046000,2025-07-02 08:37:26 PM
5,4815,Nigeria,visit_trainers,Mia Carter,1758111558000,2025-09-17 01:19:18 PM
6,8126,Nigeria,visit_home,NaN,1757215681000,2025-09-07 04:28:01 AM
7,4392,South Africa,visit_home,NaN,1748979867000,2025-06-03 08:44:27 PM
8,8204,Nigeria,visit_home,NaN,1759328534000,2025-10-01 03:22:14 PM
9,1774,South Africa,visit_plans,NaN,1757928355000,2025-09-15 10:25:55 AM


In [17]:
#Inspect df
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             9000 non-null   int64 
 1   country             9000 non-null   object
 2   user_action         9000 non-null   object
 3   trainer_name        681 non-null    object
 4   timestamp           9000 non-null   int64 
 5   readable_timestamp  9000 non-null   object
dtypes: int64(2), object(4)
memory usage: 422.0+ KB


(9000, 6)

In [18]:
#Checking for missing data or Duplicates

#missing data
print(df.isnull().sum())

#duplicates
print(f'Duplicates: {df.duplicated().sum()}')

user_id                  0
country                  0
user_action              0
trainer_name          8319
timestamp                0
readable_timestamp       0
dtype: int64
Duplicates: 0


In [19]:
print(df['country'].value_counts())
print(df['user_action'].value_counts())


country
Nigeria         2912
South Africa    2384
Egypt           1678
Ghana           1152
Kenya            538
Tanzania         336
Name: count, dtype: int64
user_action
visit_home          6656
visit_plans          923
visit_trainers       681
view_plan            303
sent_message         191
subscribed_basic     153
subscribed_pro        49
subscribed_elite      44
Name: count, dtype: int64


In [20]:
# 1) Make sure column is string and do quick cleaning
df['readable_timestamp'] = df['readable_timestamp'].astype(str).str.strip()
# remove trailing dots or accidental characters like '.' at end
df['readable_timestamp'] = df['readable_timestamp'].str.replace(
    r'\.+$', '', regex=True)
# remove duplicate spaces
df['readable_timestamp'] = df['readable_timestamp'].str.replace(
    r'\s+', ' ', regex=True)

# 2) Attempt parsing with a list of likely formats
formats_to_try = [
    "%y-%m-%d %I:%M:%S %p",   # e.g. 25-05-25 05:17:31 AM
    "%Y-%m-%d %I:%M:%S %p",   # e.g. 2025-05-25 05:17:31 AM
    "%m/%d/%Y, %I:%M %p",     # e.g. 10/29/2025, 09:45 AM
    "%m/%d/%y, %I:%M %p",     # e.g. 10/29/25, 09:45 AM
    "%y-%m-%d %H:%M:%S",      # fallback 24-hour without AM/PM
    "%Y-%m-%d %H:%M:%S"
]

# initialize a column of NaT datetimes
df['readable_dt'] = pd.NaT

for fmt in formats_to_try:
    mask = df['readable_dt'].isna()
    try:
        parsed = pd.to_datetime(
            df.loc[mask, 'readable_timestamp'], format=fmt, errors='coerce')
        df.loc[mask, 'readable_dt'] = parsed
    except Exception:
        # if a format causes unexpected error, skip it
        pass

# 3) Final fallback: let pandas infer format for remaining NaT rows
mask = df['readable_dt'].isna()
if mask.any():
    df.loc[mask, 'readable_dt'] = pd.to_datetime(
        df.loc[mask, 'readable_timestamp'], errors='coerce', infer_datetime_format=True)

# 4) Inspect how many failed to parse
failed = df['readable_dt'].isna().sum()
print(f"Parsing failures (still NaT): {failed} out of {len(df)}")

# Optionally show some problematic examples (up to 10)
if failed:
    print(df.loc[df['readable_dt'].isna(),
          'readable_timestamp'].head(10).to_list())

# 5) Create separate columns once parsed
# If you want date as ISO (YYYY-MM-DD) for Power BI, use dt.date or dt.strftime
df['date_only'] = df['readable_dt'].dt.date
df['time_only'] = df['readable_dt'].dt.strftime('%I:%M:%S %p')  # keeps AM/PM
df['hour'] = df['readable_dt'].dt.hour
df['day'] = df['readable_dt'].dt.day_name()
df['month'] = df['readable_dt'].dt.month_name()

Parsing failures (still NaT): 0 out of 9000


In [ ]:
# Export cleaned data
cleaned_path = '../data/gym_rum_data_cleaned.csv'
df.to_csv(cleaned_path, index=False)
print('Cleaned data saved:', cleaned_path)

Cleaned data saved: ../data/gym_rum_data_cleaned.csv


: 